In [3]:
using Pkg
Pkg.activate(".")

  Activating project at `~/.julia/dev/MOGMOG`


In [4]:
using MOGMOG

In [5]:
logpdf_MOG()

MethodError: MethodError: no method matching logpdf_MOG()
The function `logpdf_MOG` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  logpdf_MOG(!Matched::Float64, !Matched::Vector{Float64}, !Matched::Vector{Float64}, !Matched::Vector{Float64})
   @ MOGMOG ~/.julia/dev/MOGMOG/src/pdf.jl:3
